In [ ]:
%matplotlib inline
import os
import numpy as np
from PIL import Image
import matplotlib
from matplotlib import pyplot,pylab
plt = pyplot
import scipy
from __future__ import division
import seaborn as sns
sns.set_style('white')
import string
import pandas as pd

## purpose

* upload sketches to S3
* build stimulus dictionary and write to database

## upload sketches to s3

In [ ]:
upload_dir = './sketch'

In [ ]:
import boto
runThis = 0
if runThis:
    conn = boto.connect_s3()
    b = conn.create_bucket('sketchpad_basic_pilot2_sketches')
    all_files = [i for i in os.listdir(upload_dir) if i != '.DS_Store']
    for a in all_files:
        print a
        k = b.new_key(a)
        k.set_contents_from_filename(os.path.join(upload_dir,a))
        k.set_acl('public-read')

## build stimulus dictionary

In [ ]:
## read in experimental metadata file
path_to_metadata = '../../analysis/sketchpad_basic_pilot2_group_data.csv'
meta = pd.read_csv(path_to_metadata)

In [ ]:
## clean up and add filename column
meta2 = meta.drop(['svg','png','Unnamed: 0'],axis=1)
filename = []
games = []
for i,row in meta2.iterrows():
    filename.append('gameID_{}_trial_{}.png'.format(row['gameID'],row['trialNum']))
    games.append([])
meta2['filename'] = filename
meta2['games'] = games

In [ ]:
## write out metadata to json file
stimdict = meta2.to_dict(orient='records')
import json
with open('sketchpad_basic_recog_meta.js', 'w') as fout:
    json.dump(stimdict, fout)

In [ ]:
J = json.loads(open('sketchpad_basic_recog_meta.js',mode='ru').read())
assert len(J)==len(meta2)

In [ ]:
'{} unique games.'.format(len(np.unique(meta2.gameID.values)))

### upload stim dictionary to mongo (db = 'stimuli', collection='sketchpad_basic_recog')

In [ ]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')

In [ ]:
db = conn['stimuli']
coll = db['sketchpad_basic_pilot2_sketches']

In [ ]:
## actually add data now to the database
for (i,j) in enumerate(J):
    if i%100==0:
        print ('%d of %d' % (i,len(J)))
    coll.insert_one(j)

In [ ]:
## How many sketches have been retrieved at least once? equivalent to: coll.find({'numGames':{'$exists':1}}).count()
coll.find({'numGames':{'$gte':0}}).count()

In [ ]:
## stashed away handy querying things

# coll.find({'numGames':{'$gte':1}}).sort('trialNum')[0]

# from bson.objectid import ObjectId
# coll.find({'_id':ObjectId('5a9a003d47e3d54db0bf33cc')}).count()

### crop 3d objects 


In [ ]:
import os
from PIL import Image

def RGBA2RGB(image, color=(255, 255, 255)):
    """Alpha composite an RGBA Image with a specified color.

    Simpler, faster version than the solutions above.

    Source: http://stackoverflow.com/a/9459208/284318

    Keyword Arguments:
    image -- PIL RGBA Image object
    color -- Tuple r, g, b (default 255, 255, 255)

    """
    image.load()  # needed for split()
    background = Image.new('RGB', image.size, color)
    background.paste(image, mask=image.split()[3])  # 3 is the alpha channel
    return background

def load_and_crop_image(path, dest='object_cropped', imsize=224):
    im = Image.open(path)
#     if np.array(im).shape[-1] == 4:
#         im = RGBA2RGB(im)
    
    # crop to sketch only
    arr = np.asarray(im)
    if len(arr.shape)==2:
        w,h = np.where(arr!=127)
    else:
        w,h,d = np.where(arr!=127) # where the image is not white        
    if len(h)==0:
        print(path)            
    xlb = min(h)
    xub = max(h)
    ylb = min(w)
    yub = max(w)
    lb = min([xlb,ylb])
    ub = max([xub,yub])            
    im = im.crop((lb, lb, ub, ub))  
    im = im.resize((imsize, imsize), Image.ANTIALIAS)
    objname = path.split('/')[-1]
    if not os.path.exists(dest):
        os.makedirs(dest)
    im.save(os.path.join(dest,objname))

In [ ]:
run_this = 0
if run_this:
    ## actually crop images now
    data_dir = './object'
    allobjs = ['./object/' + i for i in os.listdir(data_dir)]
    for o in allobjs:
        load_and_crop_image(o)

In [ ]:
run_this = 0
if run_this:
    ## rename objects in folder
    data_dir = './object'
    allobjs = [data_dir + '/' + i for i in os.listdir(data_dir) if i != '.DS_Store']
    for o in allobjs:
        if len(o.split('_'))==4:
            os.rename(o, os.path.join(data_dir, o.split('/')[-1].split('_')[2] + '.png'))